### 🧩 토픽 모델링 (Topic Modeling)
#### 🧠 LDA(Latent Dirichlet Allocation)란?
- 여러 문서 안에 숨겨진 주제(Topic) 들을 자동으로 찾아내는 생성 확률 모델
- “문서는 여러 주제의 혼합으로 되어 있고, 각 주제는 특정 단어들의 분포로 이루어진다”는 가정에 기반함.

| 구성 요소             | 설명                         |
| ----------------- | -------------------------- |
| **Document (문서)** | 여러 **토픽의 혼합**으로 구성된 하나의 문서 |
| **Topic (주제)**    | 여러 **단어들의 확률 분포**로 표현됨     |
| **Word (단어)**     | 특정 **토픽**으로부터 생성된 단어       |


| 구성 요소             | 설명                         |
| ----------------- | -------------------------- |
| **Document (문서)** | 여러 **토픽의 혼합**으로 구성된 하나의 문서 |
| **Topic (주제)**    | 여러 **단어들의 확률 분포**로 표현됨     |
| **Word (단어)**     | 특정 **토픽**으로부터 생성된 단어       |


#### ⚙️ LDA 동작 원리 (간단 흐름)
1. 모든 문서를 여러 토픽의 혼합으로 가정
2. 각 토픽은 단어의 확률 분포를 가짐
3. 문서마다 토픽 비율과 단어 분포를 무작위로 초기화
4. 반복적으로 아래 과정 수행
5. 각 단어가 어떤 토픽에서 나왔는지 추정
6. 문서의 토픽 비율 업데이트
7. 토픽별 단어 분포 업데이트
8. 수렴하면 → 문서별 주요 토픽, 토픽별 주요 단어가 도출됨

In [7]:
%pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
from konlpy.tag import Okt
import konlpy

documents = [
    "Python 프로그래밍은 매우 강력하고 배우기 쉬워요",
    "머신러닝은 인공지능의 핵심 기술입니다",
    "자연어처리는 NLP라고도 불립니다",
    "딥러닝은 신경망을 이용한 학습 방법입니다",
    "데이터 분석은 통계학에 기반합니다",
    "파이썬으로 머신러닝 모델을 만들 수 있습니다",
    "인공지능 기술은 빠르게 발전하고 있습니다",
    "자연어처리 모델은 텍스트를 이해할 수 있어요"
]

# 단어 벡터화
cv = CountVectorizer(
    max_features=50,
    stop_words = ['은','는','이','가','을','를','그','그리고'],
    min_df=1,
    max_df=0.9
)
doc_term_matrix = cv.fit_transform(documents)
feature_names = cv.get_feature_names_out()
print(f'문서 단어 행렬 : {doc_term_matrix.shape}')
print(f'단어 목록 : {feature_names}  /  단어 목록 개수 : {len(feature_names)}')

문서 단어 행렬 : (8, 36)
단어 목록 : ['nlp라고도' 'python' '강력하고' '기반합니다' '기술은' '기술입니다' '데이터' '딥러닝은' '만들' '매우'
 '머신러닝' '머신러닝은' '모델은' '모델을' '발전하고' '방법입니다' '배우기' '분석은' '불립니다' '빠르게' '쉬워요'
 '신경망을' '이용한' '이해할' '인공지능' '인공지능의' '있습니다' '있어요' '자연어처리' '자연어처리는' '텍스트를'
 '통계학에' '파이썬으로' '프로그래밍은' '학습' '핵심']  /  단어 목록 개수 : 36


In [9]:
# LDA 모델 생성
from sklearn.decomposition import LatentDirichletAllocation

lda_model =  LatentDirichletAllocation(
    n_components=3  # 토픽(주제) 개수
    ,random_state=42
    ,max_iter=20
    ,learning_method='online'   # batch (모든데이터를 한번에 다써서 한번학습), online(미니배치)
)
# 모델 학습
lda_output = lda_model.fit_transform(doc_term_matrix)
print(f'문서-주제 행렬 : {lda_output.shape}')
print(f'첫번째 문서의 주제 분포')
print(f'Topic 0: {lda_output[0,0]:.4f}')
print(f'Topic 1: {lda_output[0,1]:.4f}')
print(f'Topic 2: {lda_output[0,2]:.4f}')

# 각 주제별로 상위 단어 출력
def display_topic(model, feature_nams, n_top_words = 5):
    print(f'각 주제별 상위 단어----')
    for topic_idx, topic in enumerate(model.components_):
        # 가장 높은 가중치를 가진 단어의 인덱스 호출
        top_words_idx = topic.argsort()[-n_top_words:][::-1]
        top_words = [ feature_nams[i] for i in top_words_idx]
        top_weights = [ topic[i] for i in top_words_idx ]
        print(f'[topic {topic_idx}]')
        for word,weight in zip(top_words,top_weights):
            print(f'  {word} : {weight:.4f}')
        print()
display_topic(lda_model, feature_names,n_top_words=5)

문서-주제 행렬 : (8, 3)
첫번째 문서의 주제 분포
Topic 0: 0.0481
Topic 1: 0.0483
Topic 2: 0.9036
각 주제별 상위 단어----
[topic 0]
  기술은 : 1.3185
  모델은 : 1.3178
  핵심 : 1.3125
  인공지능 : 1.3124
  기술입니다 : 1.3116

[topic 1]
  있습니다 : 1.3128
  모델을 : 1.3109
  데이터 : 1.3104
  분석은 : 1.3095
  머신러닝 : 1.3091

[topic 2]
  매우 : 1.3191
  강력하고 : 1.3182
  python : 1.3135
  nlp라고도 : 1.3131
  배우기 : 1.3099



In [10]:
lda_output[0,2]

np.float64(0.9036180312192277)

In [11]:
# 각 문서의 주요 주제
print(f'각 문서의 중요 주제 - - - - - - - - -')
for doc_idx , doc in enumerate(lda_output):
    main_topic = np.argmax(doc)
    confidence = doc[main_topic]
    print(f'문서 : {doc_idx} topic : {main_topic} 비율 : {confidence:.4f}')
    print(f'원문 : {documents[doc_idx]}')

각 문서의 중요 주제 - - - - - - - - -
문서 : 0 topic : 2 비율 : 0.9036
원문 : Python 프로그래밍은 매우 강력하고 배우기 쉬워요
문서 : 1 topic : 0 비율 : 0.8650
원문 : 머신러닝은 인공지능의 핵심 기술입니다
문서 : 2 topic : 2 비율 : 0.8312
원문 : 자연어처리는 NLP라고도 불립니다
문서 : 3 topic : 2 비율 : 0.8876
원문 : 딥러닝은 신경망을 이용한 학습 방법입니다
문서 : 4 topic : 1 비율 : 0.8655
원문 : 데이터 분석은 통계학에 기반합니다
문서 : 5 topic : 1 비율 : 0.8867
원문 : 파이썬으로 머신러닝 모델을 만들 수 있습니다
문서 : 6 topic : 0 비율 : 0.8855
원문 : 인공지능 기술은 빠르게 발전하고 있습니다
문서 : 7 topic : 0 비율 : 0.8876
원문 : 자연어처리 모델은 텍스트를 이해할 수 있어요
